<a href="https://colab.research.google.com/github/Vasilis-Kyriakopoulos/Finetune-LLM-/blob/main/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from datasets import load_dataset
# import pandas as pd


# ds = load_dataset("maxscha/commitbench")
# main_path = "data/"

# cols = ["diff", "message"]


# ds["train"].select_columns(cols) \
#     .to_pandas() \
#     .to_csv(main_path+"commitbench_train.csv", index=False)


# ds["validation"].select_columns(cols) \
#     .to_pandas() \
#     .to_csv(main_path+"commitbench_validation.csv",index=False)


# ds["test"].select_columns(cols) \
#     .to_pandas() \
#     .to_csv(main_path+"commitbench_test.csv", index=False)


In [48]:
#from google.colab import drive
import pandas as pd


main_path = "data/"
train_df = pd.read_csv(main_path + "commitbench_train.csv")
val_df = pd.read_csv(main_path + "commitbench_validation.csv")
print(f"Length Train:{len(train_df)}")
print(f"Length Val:{len(val_df)}")

train_df = train_df[train_df["diff"].str.len() < 1000].reset_index(drop=True)
val_df = val_df[val_df["diff"].str.len() < 1000].reset_index(drop=True)
train_df = train_df[~train_df['message'].str.contains('^Fixes #', na=False)]

print(f"New Length Train: {len(train_df)}")
print(f"New Length Val: {len(val_df)}")

Length Train:1165213
Length Val:249689
New Length Train: 922727
New Length Val: 197931


In [ ]:
# import pandas as pd

# input_csv = main_path + "commitbench_validation.csv"
# output_csv = main_path + "commitbench_validation_10pct.csv"

# chunk_size = 100_000
# keep_frac = 0.1
# first = True

# for chunk in pd.read_csv(input_csv, chunksize=chunk_size):
#     sampled = chunk.sample(frac=keep_frac, random_state=42)

#     sampled.to_csv(
#         output_csv,
#         mode="a",
#         index=False,
#         header=first
#     )
#     first = False


# input_csv = main_path + "commitbench_train.csv"
# output_csv = main_path + "commitbench_train_10pct.csv"

# chunk_size = 100_000
# keep_frac = 0.1
# first = True

# for chunk in pd.read_csv(input_csv, chunksize=chunk_size):
#     sampled = chunk.sample(frac=keep_frac, random_state=42)

#     sampled.to_csv(
#         output_csv,
#         mode="a",
#         index=False,
#         header=first
#     )
#     first = False


In [49]:
train_df.head()

,diff,message
0,diff --git a/lib/bibreformat.py b/lib/bibrefor...,BibFormat: HDREF processing bug fix\n\n* Fixes...
1,diff --git a/src/lib/Supra/Controller/Pages/Tw...,Task #<I>: Block titles\nfast-fix implementati...
2,diff --git a/test/unit/serializers/cbor.spec.j...,Add additional cbor test for useTag<I>ForMaps
3,diff --git a/setup.py b/setup.py\nindex <HASH>...,[build] Add tests' requires in setup.py
4,diff --git a/lib/macho/load_commands.rb b/lib/...,map lazy/upward dylib to proper load command\n...


In [50]:
val_df.head()

,diff,message
0,diff --git a/redisson/src/main/java/org/rediss...,Fixed - Redisson cluster cannot recover if Red...
1,diff --git a/translate/coroutines.py b/transla...,bug fix when transliteration doesnt exist prin...
2,diff --git a/setup.py b/setup.py\nindex <HASH>...,add feature_selection to setup.py
3,diff --git a/src/resources/views/admin/_index....,.btn outside of title h1
4,diff --git a/Db.php b/Db.php\nindex <HASH>..<H...,Accept empty condition in smartSelect


In [29]:
import numpy as np
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

def format_input(entry):

    input_text = f"{entry['diff']}\nCommit Message:\n{entry['message']}"

    return input_text

def token_len(t: str) -> int:
    return len(tokenizer.encode(t, add_special_tokens=True))


rng = np.random.default_rng(42)
texts = train_df.apply(format_input, axis=1).tolist()
sample_size = min(50_000, len(texts))
sample_texts = rng.choice(texts, size=sample_size, replace=False)

lengths = np.array([token_len(t) for t in sample_texts], dtype=np.int32)

p = [50, 75, 90, 95, 97, 98, 99, 99.5]
qs = np.percentile(lengths, p)

for perc, q in zip(p, qs):
    print(f"{perc:>5}%  -> {int(q)} tokens")

# Example choice:
max_len = int(np.percentile(lengths, 95))
print("Suggested max_len:", max_len)


   50%  -> 293 tokens
   75%  -> 363 tokens
   90%  -> 423 tokens
   95%  -> 458 tokens
   97%  -> 479 tokens
   98%  -> 496 tokens
   99%  -> 524 tokens
 99.5%  -> 550 tokens
Suggested max_len: 458


In [51]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import re

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(tokenizer.encode("<|endoftext|>"))
print(train_df.iloc[0])
print(format_input(train_df.iloc[0]))

def clean_text(text):
    # 1. Remove SHA-1 hashes (usually 7-40 hex characters)
    text = re.sub(r'\b[0-9a-f]{7,40}\b', '', text)
    
    # 2. Remove Git metadata like "Signed-off-by" or "Co-authored-by"
    text = re.sub(r'^(Signed-off-by|Co-authored-by|Reported-by):.*$', '', text, flags=re.MULTILINE)
    
    # 3. Remove excessive newlines and whitespace
    text = re.sub(r'\n\s*\n', '\n', text)

    # 1. Αφαίρεση της γραμμής 'index <hash>..<hash> <mode>'
    text = re.sub(r'^index [0-9a-fA-F]+\.\.[0-9a-fA-F]+ \d+\n', '', text, flags=re.MULTILINE)
    
    # 2. Απλοποίηση του 'diff --git a/path/to/file b/path/to/file' 
    # σε 'FILE: path/to/file' για εξοικονόμηση tokens
    text = re.sub(r'^diff --git a/(.*) b/(.*)\n', r'FILE: \1\n', text, flags=re.MULTILINE)
    
    # 3. Αφαίρεση των γραμμών --- a/ και +++ b/ (είναι πλεονασμός αφού έχουμε το FILE:)
    text = re.sub(r'^--- a/.*\n', '', text, flags=re.MULTILINE)
    text = re.sub(r'^\+\+\+ b/.*\n', '', text, flags=re.MULTILINE)
    return text.strip()


class CodeDiffMessageDataset(Dataset):
    def __init__(self, data, tokenizer,max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = 512


    def __getitem__(self, index):
        entry = self.data.iloc[index]
        
        # Clean the components
        clean_diff = clean_text(entry['diff'])
        clean_msg = clean_text(entry['message'])

        # Structure: <|endoftext|> DIFF: ... MESSAGE: ... <|endoftext|>
        prompt = f"<|endoftext|>DIFF:\n{clean_diff}\n\nCOMMIT MESSAGE:\n"
        full_text = f"{prompt}{clean_msg}<|endoftext|>"

        encoded_full = self.tokenizer.encode(full_text, max_length=self.max_length, truncation=True)
        encoded_prompt = self.tokenizer.encode(prompt, max_length=self.max_length, truncation=True)
        
        return  encoded_full


    def __len__(self):
        return int(len(self.data)/10)

train_dataset = CodeDiffMessageDataset(train_df, tokenizer)

def custom_collate_fn(
    batch,
    pad_token_id=50256,
    ignore_index=-100,
):
    # Find the longest sequence in the batch
    batch_max_length = max(len(item)+1 for item in batch)

    # Pad and prepare inputs and targets
    inputs_lst, targets_lst = [], []

    for item in batch:
        new_item = item.copy()
        # Add an <|endoftext|> token
        new_item += [pad_token_id]
        # Pad sequences to max_length
        padded = (
            new_item + [pad_token_id] *
            (batch_max_length - len(new_item))
        )
        inputs = torch.tensor(padded[:-1])  # Truncate the last token for inputs
        targets = torch.tensor(padded[1:])  # Shift +1 to the right for targets

        # New: Replace all but the first padding tokens in targets by ignore_index
        mask = targets == pad_token_id
        indices = torch.nonzero(mask).squeeze()
        if indices.numel() > 1:
            targets[indices[1:]] = ignore_index


        inputs_lst.append(inputs)
        targets_lst.append(targets)

    # Convert list of inputs and targets to tensors and transfer to target device
    device = None
    if torch.cuda.is_available():
      device = torch.device("cuda")
    else:
      device = torch.device("cpu")
    inputs_tensor = torch.stack(inputs_lst).to(device)
    targets_tensor = torch.stack(targets_lst).to(device)

    return inputs_tensor, targets_tensor


if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Device:", device)

[50256]
diff       diff --git a/lib/bibreformat.py b/lib/bibrefor...
message    BibFormat: HDREF processing bug fix\n\n* Fixes...
Name: 0, dtype: object
diff --git a/lib/bibreformat.py b/lib/bibreformat.py
index <HASH>..<HASH> 100644
--- a/lib/bibreformat.py
+++ b/lib/bibreformat.py
@@ -126,7 +126,7 @@ def bibreformat_task(fmt, sql, sql_queries, cds_query, process_format, process, 
         write_message("Querying database (%s) ..." % sql_query, verbose=2)
         recIDs |= intbitset(run_sql(sql_query))
 
-    if fmt == "HDREF":
+    if fmt == "HDREF" and recIDs:
         # HDREF represents the references tab
         # the tab needs to be recomputed not only when the record changes
         # but also when one of the citations changes
Commit Message:
BibFormat: HDREF processing bug fix

* Fixes bug that would occur with empty records when processing HDREF.
  (closes #<I>)
Device: cuda


In [31]:
inputs_1 = [0, 1, 2, 3, 4]
inputs_2 = [5, 6]
inputs_3 = [7, 8, 9]

batch = (
    inputs_1,
    inputs_2,
    inputs_3
)

print(custom_collate_fn(batch))

(tensor([[    0,     1,     2,     3,     4],
        [    5,     6, 50256, 50256, 50256],
        [    7,     8,     9, 50256, 50256]], device='cuda:0'), tensor([[    1,     2,     3,     4, 50256],
        [    6, 50256,  -100,  -100,  -100],
        [    8,     9, 50256,  -100,  -100]], device='cuda:0'))


In [52]:
from torch.utils.data import DataLoader


num_workers = 0
batch_size = 4

torch.manual_seed(123)

train_dataset = CodeDiffMessageDataset(train_df, tokenizer)
val_dataset = CodeDiffMessageDataset(val_df, tokenizer)
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    collate_fn=custom_collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    collate_fn=custom_collate_fn,
    shuffle= False,
    drop_last=True,
    num_workers=num_workers
)

In [38]:
print("Train loader:")
for inputs, targets in train_loader:
    print(inputs.shape, targets.shape)

Train loader:
torch.Size([4, 490]) torch.Size([4, 490])
torch.Size([4, 411]) torch.Size([4, 411])
torch.Size([4, 297]) torch.Size([4, 297])
torch.Size([4, 372]) torch.Size([4, 372])
torch.Size([4, 318]) torch.Size([4, 318])
torch.Size([4, 324]) torch.Size([4, 324])
torch.Size([4, 365]) torch.Size([4, 365])
torch.Size([4, 394]) torch.Size([4, 394])
torch.Size([4, 441]) torch.Size([4, 441])
torch.Size([4, 447]) torch.Size([4, 447])
torch.Size([4, 386]) torch.Size([4, 386])
torch.Size([4, 352]) torch.Size([4, 352])
torch.Size([4, 469]) torch.Size([4, 469])
torch.Size([4, 315]) torch.Size([4, 315])
torch.Size([4, 333]) torch.Size([4, 333])
torch.Size([4, 447]) torch.Size([4, 447])
torch.Size([4, 368]) torch.Size([4, 368])
torch.Size([4, 466]) torch.Size([4, 466])
torch.Size([4, 444]) torch.Size([4, 444])
torch.Size([4, 273]) torch.Size([4, 273])
torch.Size([4, 460]) torch.Size([4, 460])
torch.Size([4, 381]) torch.Size([4, 381])
torch.Size([4, 345]) torch.Size([4, 345])
torch.Size([4, 255])

In [37]:
# 1. Παίρνουμε το πρώτο batch
first_batch = next(iter(train_loader))

# 2. Αποσυμπλέκουμε τα δεδομένα (input_ids και labels)
input_ids, labels = first_batch

print(f"Batch Input Shape: {input_ids.shape}")
print(f"Batch Labels Shape: {labels.shape}")

# 3. Δες το πρώτο δείγμα του batch σε μορφή κειμένου
sample_index = 0
decoded_text = tokenizer.decode(input_ids[sample_index], skip_special_tokens=False)

print("\n--- Πρώτο Δείγμα (Decoded) ---")
print(input_ids[sample_index])

print("\n--- Labels (για να δεις το Masking με -100) ---")
print(labels[sample_index])

Batch Input Shape: torch.Size([4, 399])
Batch Labels Shape: torch.Size([4, 399])

--- Πρώτο Δείγμα (Decoded) ---
tensor([50256,    35, 29267,    25,   198, 26069,  1377, 18300,   257,    14,
        16469,    12, 18769,    12, 16302,    14, 16469,    12, 18769,    14,
        10677,    14, 12417,    14, 12355,    14,  2398,    14, 16469, 30604,
           14, 18769,    14, 12384,    14, 16366,    14, 19452, 30800, 32875,
           13, 12355,   275,    14, 16469,    12, 18769,    12, 16302,    14,
        16469,    12, 18769,    14, 10677,    14, 12417,    14, 12355,    14,
         2398,    14, 16469, 30604,    14, 18769,    14, 12384,    14, 16366,
           14, 19452, 30800, 32875,    13, 12355,   198,  9630,  1279,    39,
        11211,    29,   492,    27,    39, 11211,    29,  1802, 29173,   198,
         6329,   257,    14, 16469,    12, 18769,    12, 16302,    14, 16469,
           12, 18769,    14, 10677,    14, 12417,    14, 12355,    14,  2398,
           14, 16469, 30604, 

In [ ]:

print(targets[5])

In [55]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

model.config.pad_token_id = tokenizer.eos_token_id
model.config.use_cache = False
model.config.pad_token_id = model.config.eos_token_id
print(tokenizer.vocab_size)
model.config.pad_token_id





50257


50256

In [39]:
print(model.config)

GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "dtype": "float32",
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.57.6",
  "use_cache": false,
  "vocab_size": 50257
}



In [ ]:
import torch
import torch.nn as nn
from tqdm.auto import tqdm
import math
from transformers import get_linear_schedule_with_warmup

def calculate_accuracy(logits, labels):
    predictions = torch.argmax(logits, dim=-1) #(batch_size, sequence_length, vocab_size)
    correct_predictions = (predictions == labels).float() # (batch_size, sequence_length)
    return correct_predictions.mean().item()

def get_random_validation_diff(val_df):
    # Διαλέγουμε μια τυχαία γραμμή από το validation dataframe
    random_row = val_df.sample(n=1).iloc[0]
    return random_row['diff'], random_row['message']

def generate(model, tokenizer, device,val_df):
    model.eval()
    val_diff, actual_message = get_random_validation_diff(val_df)
    prompt = f"<|endoftext|>DIFF:\n{val_diff}\n\nCOMMIT MESSAGE:\n"
    inputs = tokenizer(
        prompt, 
        return_tensors="pt",
        return_att 
        max_length=512, 
        truncation=True
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=40,
            do_sample=True,      
            top_p=0.9,
            repetition_penalty=1.2, 
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # Αποκωδικοποίηση μόνο της απάντησης
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_message = full_text.split("COMMIT MESSAGE:")[-1].strip()
    
    print("\n" + "="*50)
    print(f"TEST ON RANDOM VAL DIFF:")
    print(f"DIFF (Truncated):\n{val_diff[:200]}...")
    print(f"\nACTUAL MESSAGE: {actual_message}")
    print(f"GENERATED MESSAGE: {generated_message}")
    print("="*50 + "\n")
    model.train()

# --- 2. Training Step ---
def train_one_epoch(model, dataloader, optimizer, device, progress_bar, tokenizer,val_df):
    model.train()
    total_loss = 0
    total_acc = 0

    for i, (x, y) in enumerate(dataloader):
        
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        # Forward pass
        logits = model(x).logits
        loss = nn.CrossEntropyLoss()(
            logits.view(-1, logits.size(-1)),
            y.view(-1)
        )
        acc = calculate_accuracy(logits, y)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        total_acc += acc

        progress_bar.update(1)
        progress_bar.set_postfix({"train_loss": f"{loss.item():.4f}","acc": f"{acc:.4f}","lr": f"{scheduler.get_last_lr()[0]:.2e}"})


        if i > 0 and i % 100 == 0:
            generate(model, tokenizer, device,val_df)

    avg_loss = total_loss / len(dataloader)
    avg_acc = total_acc / len(dataloader)
    return avg_loss,avg_acc

# --- 3. Evaluation Step ---
def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    total_acc=0
    with torch.no_grad():

        for x, y in dataloader:
            x = x.to(device)
            y = y.to(device)

            logits = model(x).logits
            loss = nn.CrossEntropyLoss()(
                logits.view(-1, logits.size(-1)),
                y.view(-1)
            )
            total_loss += loss.item()
            total_acc  += calculate_accuracy(logits, y)


    avg_loss = total_loss / len(dataloader)
    avg_acc = total_acc / len(dataloader)
    return avg_loss,avg_acc


# --- 4. Main Loop ---
def train(model, train_loader, val_loader, optimizer,scheduler, device, tokenizer,epochs,val_df):
    best_val_loss = float("inf")
    patience = 2
    bad_epochs = 0


    for epoch in range(epochs):

        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}")

        # Train
        train_loss,train_acc = train_one_epoch(model, train_loader, optimizer, device, progress_bar, tokenizer,val_df)

        # Evaluate
        val_loss,val_acc = evaluate(model, val_loader, device)
        try:
            ppl = math.exp(val_loss)
        except OverflowError:
            ppl = float('inf')

        progress_bar.set_postfix({
            "train_loss": f"{train_loss:.4f}",
            "train_accuracy": f"{train_acc:.4f}",
            "val_loss": f"{val_loss:.4f}",
            "val_accuracy": f"{val_acc:.4f}",
            "PPL": f"{ppl:.2f}"
        })
        progress_bar.refresh()
        progress_bar.close() # Close bar to print new line

        # Logging
        print(f"Summary Epoch {epoch+1}: Train Loss: {train_loss:.4f}| Train Accuracy: {train_acc:.4f} \
        |Val Loss: {val_loss:.4f}|Val Accuracy: {val_acc:.4f})| PPL: {ppl:.4f}")

        # Save checkpoints
        torch.save(model.state_dict(), f"{main_path}/gpt2_epoch{epoch+1}.pt")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            bad_epochs = 0
            torch.save(model.state_dict(), f"{main_path}/best_model.pt")
            print(">>> New best model saved!")
        else:
            bad_epochs += 1
            print(f">>> No improvement (bad epochs: {bad_epochs})")

        if bad_epochs >= patience:
            print("EARLY STOPPING TRIGGERED.")
            break

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
lr = 5e-5
epochs = 3
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
num_training_steps = num_epochs * len(train_loader)
num_warmup_steps = int(0.1 * num_training_steps) 
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=num_warmup_steps, 
    num_training_steps=num_training_steps
)
train(model, train_loader, val_loader, optimizer, scheduler ,device, tokenizer,epochs,val_df)

Epoch 1/3:   0%|          | 0/23068 [00:00<?, ?it/s]


TEST ON RANDOM VAL DIFF:
DIFF (Truncated):
diff --git a/stryker.conf.js b/stryker.conf.js
index <HASH>..<HASH> 100644
--- a/stryker.conf.js
+++ b/stryker.conf.js
@@ -23,9 +23,7 @@ module.exports = function(config) {
     },
     mutate: [
    ...

ACTUAL MESSAGE: CA-<I> Adding back in mutation for models
GENERATED MESSAGE: -- ( ) .md is not used here because this file uses it instead of the `modules` directive; but if you want to use that format in your CSS files then there's no need for


TEST ON RANDOM VAL DIFF:
DIFF (Truncated):
diff --git a/python/ray/rllib/agents/ars/ars.py b/python/ray/rllib/agents/ars/ars.py
index <HASH>..<HASH> 100644
--- a/python/ray/rllib/agents/ars/ars.py
+++ b/python/ray/rllib/agents/ars/ars.py
@@ -2...

ACTUAL MESSAGE: Ars increase (#<I>)

* removed cv2



* remove opencv



* increased number of default rollouts ARS



* put cv2 back in this branch



* put cv2 back in this branch



* moved cv2 back where it belongs in preprocessors
GENERATED MESSAGE: "

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. Ορισμός συσκευής και μοντέλου βάσης
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "gpt2"

# 2. Φόρτωση του tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 3. Αρχικοποίηση του μοντέλου (Base GPT-2)
model = AutoModelForCausalLM.from_pretrained(model_name)

# 4. Φόρτωση των δικών σου βαρών από το .pt αρχείο
checkpoint_path = 'data/best_model.pt'
state_dict = torch.load(checkpoint_path, map_location=device)

# Φόρτωση των βαρών στο μοντέλο
model.load_state_dict(state_dict)

model.to(device)
model.eval()

print("Το εκπαιδευμένο μοντέλο φορτώθηκε επιτυχώς!")

Το εκπαιδευμένο μοντέλο φορτώθηκε επιτυχώς!


In [25]:
def generate_commit_message(diff_text, model, tokenizer, device):
    model.eval()
    
    # 1. Καθαρισμός και σωστό Format (ίδιο με το training!)
    clean_diff = (diff_text) 
    prompt = f"<|endoftext|>DIFF:\n{clean_diff}\n\nCOMMIT MESSAGE:\n"
    
    # 2. Tokenization
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # 3. Παραγωγή (Generation)
    with torch.no_grad():
        output_tokens = model.generate(
            **inputs,
            max_new_tokens=50,      
            do_sample=True,        
            top_p=0.92,             
            temperature=0.7,        
            pad_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=2  
        )
    
    # 4. Decoding
    full_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    
    # Επιστρέφουμε μόνο το κομμάτι μετά το "COMMIT MESSAGE:"
    message = full_text.split("COMMIT MESSAGE:")[-1].strip()
    return message

test_diff = """--- a/app.py
+++ b/app.py
- def calculate(a, b): return a+b
+ def add_numbers(a, b): return a + b"""
print(generate_commit_message(test_diff,model,tokenizer,device))
    

Increment numbers to use numbers instead of numbers for number of units.


In [11]:
import torch
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. Setup the App
app = FastAPI(title="Git Diff to Commit Message API")

# 2. Define Request Schema
# This ensures users send valid JSON with a "diff" field
class DiffRequest(BaseModel):
    diff: str
    max_tokens: int = 50  # Default value, but user can change it

# 3. Global Variables for Model (Loaded on Startup)
model = None
tokenizer = None
device = None

@app.on_event("startup")
def load_model():
    global model, tokenizer, device

    # Configuration
    MODEL_PATH = "best_model.pt" # Or your specific checkpoint path
    BASE_MODEL_NAME = "gpt2"     # Needed for the tokenizer configuration

    print("Loading model and tokenizer...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load Tokenizer (Use the base gpt2 tokenizer)
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token

    # Load Model Structure & Weights
    model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_NAME)
    model.resize_token_embeddings(len(tokenizer)) # Important if you resized during training

    # Load your fine-tuned weights
    # map_location ensures it loads even if you move from GPU -> CPU
    state_dict = torch.load(MODEL_PATH, map_location=device)
    model.load_state_dict(state_dict)

    model.to(device)
    model.eval()
    print(f"Model loaded successfully on {device}!")

# 4. Generation Endpoint
@app.post("/generate")
async def generate_commit_message(request: DiffRequest):
    if not model:
        raise HTTPException(status_code=500, detail="Model not loaded")

    try:
        # Format the input exactly how we trained it
        generate_commit_message(DiffRequest,model,tokenizer,device)

        return {
            "generated_message": answer,
            "full_text": full_text
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# 5. Health Check
@app.get("/health")
def health_check():
    return {"status": "ok", "device": str(device)}

/tmp/ipython-input-463976673.py:20: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [27]:
import requests

url = "http://127.0.0.1:8000/generate"
data = {"diff": "--- a/file.py\n+++ b/file.py\n- print('error')\n+ print('fixed')"}

response = requests.post(url, json=data)
print(response.json()['generated_message'])

fix(file) fix error handling
